In [1]:
import numpy as np
import csv
from itertools import product

In [2]:
def parse_csv(filepath):
    with open(filepath) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 1
        episodes = []
        for row in csv_reader:
            if line_count==1: m = int(row[0])
            elif line_count==2: num_actions = int(row[0])
            elif line_count==3: k = int(row[0])
            elif line_count==4: 
                state_rep_size = (k+1)**m
                theta_b = np.array(row, dtype=np.float).reshape(num_actions, state_rep_size).T
            elif line_count==5: n = int(row[0])
            elif line_count<=(n+5): episodes.append(np.array(row, dtype=np.float))
            else: policy_test = np.array(row, dtype=np.float)
            line_count += 1
    return m, num_actions, k, theta_b, episodes, policy_test
    
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

def pi(s,theta,c):
    phi_s = np.cos(np.pi * np.dot(c,s)).flatten()
    return softmax(np.dot(phi_s,theta))

In [3]:
m, num_actions, k, theta_b, episodes, policy_test_expected = parse_csv('data.csv')
c = np.flip(list(product(range(k+1), repeat=m)), axis=1)

print(f'num state features (m): {m}')
print(f'num actions: {num_actions}')
print(f'fourier basis order (k): {k}')
print(f'theta_b: \n{theta_b}')
print(f'num episodes: {len(episodes)}')
print(f'first episode: {episodes[0]}')

policy_test_actual = np.array([pi(state, theta_b, c) for state in episodes[0][::3]])
policy_test_actual = policy_test_actual[np.arange(policy_test_actual.shape[0]),np.array(episodes[0][1::3],dtype=int)]
print(f'policy expected: {policy_test_expected}')
print(f'policy actual: {policy_test_actual}')

num state features (m): 1
num actions: 2
fourier basis order (k): 1
theta_b: 
[[ 0.01  1.  ]
 [-0.01  1.  ]]
num episodes: 200000
first episode: [ 0.419908   1.         0.992628   0.366283   0.        10.
  0.0622811  1.         2.98566    0.327772   1.         1.69524
  0.612293   0.         6.10134  ]
policy expected: [0.775818 0.197512 0.878759 0.819091 0.345012]
policy actual: [0.77581796 0.19751227 0.87875879 0.81909146 0.34501192]
